Determining Equivalance Relative Spectral Response

In [1]:
import numpy as np
import math
from scipy.interpolate import UnivariateSpline
from scipy import signal
from scipy import ndimage
from scipy import stats
from scipy.stats import norm
import emd # From - https://github.com/andreasjansson/python-emd
import pandas
import plotly

plotly.offline.init_notebook_mode()

Read data to pandas array

In [2]:
ls8oli = pandas.read_excel('Ball_BA_RSR.xlsx',sheetname = None)
ls8tirs = pandas.read_excel('TIRS_Relative_Spectral_ResponsesBA.xlsx', sheetname = ['TIRS BA RSR'])
ls5tm = pandas.read_excel('L5_TM_RSR.xlsx',sheetname = None)
ls7etm = pandas.read_excel('L7_RSR.xlsx',sheetname = ['Blue-L7','Green-L7','Red-L7','NIR-L7', 'SWIR(5)-L7','SWIR(7)-L7','Pan-L7'])
# Band8 has some dodgy outliers i.e. bimodal with zeros between - two leading values removed in spreadsheet
s2amsi = pandas.read_excel('Sentinel-2A MSI Spectral Responses.xlsx',sheetname = ['Spectral Responses'])
tmodis = pandas.read_excel('MODIS_FM1_IB_OOB_RSR_merged.xls', sheetname = ['modis_fm1_rsp_mid_chnl_sum'])
#ASTER - http://asterweb.jpl.nasa.gov/content/01_mission/03_instrument/archive/swir.txt (vnir.txt, tir.txt)
astervnir = pandas.read_csv('aster/vnir.txt',delim_whitespace=True, header=3)
asterswir = pandas.read_csv('aster/swir.txt',delim_whitespace=True, header=3)
astertir = pandas.read_csv('aster/tir.txt',delim_whitespace=True, header=3)
#H8 - http://www.data.jma.go.jp/mscweb/en/himawari89/space_segment/spsg_ahi.html
h8ahi = pandas.read_excel('h8/AHI-8_SpectralResponsivity_Data.xlsx', sheetname = ['Band 1','Band 2','Band 3','Band 4','Band 5','Band 6','Band 7','Band 8','Band 9','Band 10','Band 11','Band 12','Band 13','Band 14','Band 15','Band 16'], header = [4])
irradiance = pandas.read_excel('solar_irradiance/ASTMG173.xls', sheetname = ['SMARTS2'], header=1)
print "MODIS:",type(tmodis),"ASTER:",type(astervnir), "Irradiance:", type(irradiance)

MODIS: <type 'dict'> ASTER: <class 'pandas.core.frame.DataFrame'> Irradiance: <type 'dict'>


In [3]:
print "ASTER VNIR", astervnir.keys()
print "ASTER TIR", astertir.keys()
print "ASTER SWIR", asterswir.keys()
print "Himawari 8 AHI", h8ahi.keys()
print "OLI ",ls8oli.keys()
print "TIRS", ls8tirs.keys()
print "TM", ls5tm.keys()
print "ETM+", ls7etm.keys()
print "S2AMSI", s2amsi.keys()
print "Terra MODIS", (tmodis.keys())
print "Solar Irradiance", (irradiance.keys())
sensorlist = [astervnir,asterswir,astertir, h8ahi,ls8oli,ls8tirs,ls5tm,ls7etm,s2amsi,tmodis,irradiance]

ASTER VNIR Index([u'[um]', u'Response', u'[um].1', u'Response.1', u'[um].2',
       u'Response.2', u'[um].3', u'Response.3'],
      dtype='object')
ASTER TIR Index([u'(um)', u'Resp', u'(um).1', u'Resp.1', u'(um).2', u'Resp.2', u'(um).3',
       u'Resp.3', u'(um).4', u'Resp.4'],
      dtype='object')
ASTER SWIR Index([u'(um)', u'Resp', u'(um).1', u'Resp.1', u'(um).2', u'Resp.2', u'(um).3',
       u'Resp.3', u'(um).4', u'Resp.4', u'(um).5', u'Resp.5'],
      dtype='object')
Himawari 8 AHI ['Band 9', 'Band 8', 'Band 3', 'Band 2', 'Band 1', 'Band 7', 'Band 6', 'Band 5', 'Band 4', 'Band 13', 'Band 12', 'Band 11', 'Band 10', 'Band 16', 'Band 15', 'Band 14']
OLI  [u'Blue', u'Cirrus', u'SWIR1', u'SWIR2', u'CoastalAerosol', u'Green', u'Pan', u'NIR', u'Red']
TIRS ['TIRS BA RSR']
TM [u'Blue-L5 TM', u'SWIR(5)-L5 TM', u'Green-L5 TM', u'Red-L5 TM', u'SWIR(7)-L5 TM', u'NIR-L5 TM']
ETM+ ['Pan-L7', 'SWIR(5)-L7', 'Blue-L7', 'Red-L7', 'NIR-L7', 'SWIR(7)-L7', 'Green-L7']
S2AMSI ['Spectral Responses']
Terr

Assign to common set of column names to make navigation simpler

In [4]:
def updatecolumns(df):
    for key in df.keys():
        if len(df[key].columns) == 3:
            df[key].columns = ['Wavelength', 'RSR', 'std']   
        if len(df[key].columns) == 2:
            df[key].columns = ['Wavelength', 'RSR']
    return(df)

ls8oli = updatecolumns(ls8oli)

for key in h8ahi.keys():
    h8ahi[key].columns = ['Wavelength', 'Wavenumber', 'RSR']
    
astervnir.columns = ['Wavelength1','VNIR_Band1','Wavelength2','VNIR_Band2','Wavelength3','VNIR_Band3', 'Wavelength3B','VNIR_Band3B']
asterswir.columns = ['Wavelength4','SWIR_Band4','Wavelength5','SWIR_Band5','Wavelength6','SWIR_Band6','Wavelength7','SWIR_Band7','Wavelength8','SWIR_Band8','Wavelength9','SWIR_Band9']
astertir.columns = ['Wavelength10','TIR_Band10','Wavelength11','TIR_Band11','Wavelength12','TIR_Band12','Wavelength13','TIR_Band13','Wavelength14','TIR_Band14']
ls8tirs['TIRS BA RSR'].columns = ['Wavelength', 'TIRS1RSR', 'TIRS2RSR', 'ignore','ignore','ignore','ignore']
ls7etm = updatecolumns(ls7etm)
ls5tm = updatecolumns(ls5tm)
s2amsi['Spectral Responses'].columns = ['Wavelength', 'Band1', 'Band2', 'Band3', 'Band4', 'Band5',
       'Band6', 'Band7', 'Band8', 'Band8A', 'Band9', 'Band10',
       'Band11', 'Band12']
tmodis['modis_fm1_rsp_mid_chnl_sum'].columns = ['Wavelength1', 'Band1RSR', 'Wavelength2', 'Band2RSR', 'Wavelength3',
       'Band3RSR', 'Wavelength4', 'Band4RSR', 'Wavelength5', 'Band5RSR',
       'Wavelength6', 'Band6RSR', 'Wavelength7', 'Band7RSR', 'Wavelength8',
       'Band8RSR', 'Wavelength9', 'Band9RSR', 'Wavelength10', 'Band10RSR',
       'Wavelength11', 'Band11RSR', 'Wavelength12', 'Band12RSR', 'Wavelength13',
       'Band13RSR', 'Wavelength14', 'Band14RSR', 'Wavelength15', 'Band15RSR',
       'Wavelength16', 'Band16RSR', 'Wavelength17', 'Band17RSR', 'Wavelength18',
       'Band18RSR', 'Wavelength19', 'Band19RSR', 'Wavelength20', 'Band20RSR',
       'Wavelength21', 'Band21RSR', 'Wavelength22', 'Band22RSR', 'Wavelength23',
       'Band23RSR', 'Wavelength24', 'Band24RSR', 'Wavelength25', 'Band25RSR',
       'Wavelength26', 'Band26RSR', 'Wavelength27', 'Band27RSR', 'Wavelength28',
       'Band28RSR', 'Wavelength29', 'Band29RSR', 'Wavelength30', 'Band30RSR',
       'Wavelength31', 'Band31RSR', 'Wavelength32', 'Band32RSR', 'Wavelength33',
       'Band33RSR', 'Wavelength34', 'Band34RSR', 'Wavelength35', 'Band35RSR',
       'Wavelength36', 'Band36RSR']
irradiance['SMARTS2'].columns = ['Wavelength', 'Power', 'ignore', 'ignore']


In [5]:
# Combine as commonly references dict of dataframes combinedsensors['sensor_band'].wavelength
# Define a sensible dataframe to hold the sensor info - above is getting blurghy
# Room to add sensor native sampling resolution too though we can consider that out of scope for now

# combinedsensors = pandas.DataFrame([{ "wavelength": , "rsr":  }], dtype = 'sensor')

# A dict of pandas frames sounds like a reasonable container for this

sensordict = {}

#TODO def modes/types

# Type 1 - dict of bands i.e. one band per sheet, one wavelength column, one rsr columns

sensor = 'ls8oli'
sensordict[sensor] = {}
inputdict = ls8oli

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

sensor = 'ls5tm'
sensordict[sensor] = {}
inputdict = ls5tm

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

sensor = 'ls7etm'
sensordict[sensor] = {}
inputdict = ls7etm

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength)
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors
    
sensor = 'h8ahi'
sensordict[sensor] = {}
inputdict = h8ahi

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors    
 
# Type 2 - single dict i.e. multiple bands per sheet, one wavelength column, multiple rsr columns 
    
sensor = 's2amsi'
sensordict[sensor] = {}
inputdict = s2amsi
for key in inputdict.keys():

    for column in inputdict[(key)].columns:

        if not (column == 'Wavelength'):
            sensorrsr = pandas.Series(inputdict[(key)][column])
            sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
            combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
            
            sensordict[sensor][(column)] = combinedsensors



In [6]:
sensor = 'ls8tirs'
sensordict[sensor] = {}
inputdict = ls8tirs
for key in inputdict.keys():
    for column in inputdict[(key)].columns:
        if not (column == 'Wavelength') and not (column == 'ignore'):
            sensorrsr = pandas.Series(inputdict[(key)][column])
            sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
            combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
            
            sensordict[sensor][(column)] = combinedsensors            

In [7]:
# Type 3 - one band in dict i.e. alternating wavelength and column  per sheet, one wavelength column, one rsr columns
# Combine all of the ASTER telescope to a single dictionary - leaving OLI and TIRS seperate for now

sensor = 'aster'
sensordict[sensor] = {}
inputdict = astervnir
sensordict[sensor][('VNIR_Band1')] = pandas.DataFrame({'wavelength': astervnir.Wavelength1*1000, 'rsr': astervnir.VNIR_Band1})
sensordict[sensor][('VNIR_Band2')] = pandas.DataFrame({'wavelength': astervnir.Wavelength2*1000, 'rsr': astervnir.VNIR_Band2})
sensordict[sensor][('VNIR_Band3')] = pandas.DataFrame({'wavelength': astervnir.Wavelength3*1000, 'rsr': astervnir.VNIR_Band3})
sensordict[sensor][('VNIR_Band3B')] = pandas.DataFrame({'wavelength': astervnir.Wavelength3B*1000, 'rsr': astervnir.VNIR_Band3B})
sensordict[sensor][('SWIR_Band4')] = pandas.DataFrame({'wavelength': asterswir.Wavelength4*1000, 'rsr': asterswir.SWIR_Band4})
sensordict[sensor][('SWIR_Band5')] = pandas.DataFrame({'wavelength': asterswir.Wavelength5*1000, 'rsr': asterswir.SWIR_Band5})
sensordict[sensor][('SWIR_Band6')] = pandas.DataFrame({'wavelength': asterswir.Wavelength6*1000, 'rsr': asterswir.SWIR_Band6})
sensordict[sensor][('SWIR_Band7')] = pandas.DataFrame({'wavelength': asterswir.Wavelength7*1000, 'rsr': asterswir.SWIR_Band7})
sensordict[sensor][('SWIR_Band8')] = pandas.DataFrame({'wavelength': asterswir.Wavelength8*1000, 'rsr': asterswir.SWIR_Band8})
sensordict[sensor][('SWIR_Band9')] = pandas.DataFrame({'wavelength': asterswir.Wavelength9*1000, 'rsr': asterswir.SWIR_Band9})
sensordict[sensor][('TIR_Band10')] = pandas.DataFrame({'wavelength': astertir.Wavelength10*1000, 'rsr': astertir.TIR_Band10})
sensordict[sensor][('TIR_Band11')] = pandas.DataFrame({'wavelength': astertir.Wavelength11*1000, 'rsr': astertir.TIR_Band11})
sensordict[sensor][('TIR_Band12')] = pandas.DataFrame({'wavelength': astertir.Wavelength12*1000, 'rsr': astertir.TIR_Band12})
sensordict[sensor][('TIR_Band13')] = pandas.DataFrame({'wavelength': astertir.Wavelength13*1000, 'rsr': astertir.TIR_Band13})
sensordict[sensor][('TIR_Band14')] = pandas.DataFrame({'wavelength': astertir.Wavelength14*1000, 'rsr': astertir.TIR_Band14})


In [59]:
# Type 4 - Solar Irradiance - supposing we normalise this or find a normalised source other than 
# http://www.astm.org/Standards/G173.htm 
sensor = 'irradiance'
sensordict[sensor] = {}
inputdict = irradiance
sensordict[sensor][('solar')] = pandas.DataFrame({'wavelength': \
                                                       irradiance['SMARTS2'].Wavelength,\
                                                       'rsr': \
                                                       irradiance['SMARTS2'].Power})
solar_curve = ndimage.filters.gaussian_filter(sensordict['irradiance'][('solar')].rsr, 20)


In [60]:
# Plot solar spectrum

plotly.offline.iplot({
"data": [{"x": sensordict['irradiance'][('solar')].wavelength,\
          "y": sensordict['irradiance'][('solar')].rsr/sensordict['irradiance'][('solar')].rsr.max(),\
          "name": 'Solar Irradiance'},
         {"x": sensordict['irradiance'][('solar')].wavelength,"y": solar_curve, "name": 'Solar Irradiance'},
        ],
"layout": {"title": "SMARTS Solar Irradiance"}})


In [11]:
# The holy grail awaits...
sensordict.keys()
#sensordict[sensor][(column)]

['irradiance',
 'ls8tirs',
 'ls8oli',
 'ls5tm',
 'aster',
 'ls7etm',
 'h8ahi',
 's2amsi',
 'tmodis']

In [12]:
# Example mixing sensordict and individual dict of dataframes access
# We want a function that enables return "sensors that are green" for example

print plotly.__version__            # version 1.9.4 required

plotly.offline.iplot({
"data": [{"x": ls8oli[('Green')].Wavelength,"y": ls8oli[('Green')].RSR, "name": 'ls8oli GREEN'},\
         {"x": ls5tm[('Green-L5 TM')].Wavelength,"y": ls5tm[('Green-L5 TM')].RSR, "name": 'ls5tm GREEN'},\
         {"x": ls7etm[('Green-L7')].Wavelength,"y": ls7etm[('Green-L7')].RSR, "name": 'ls7etm GREEN'},\
         {"x": sensordict['s2amsi'][('Band3')].wavelength,"y": sensordict['s2amsi'][('Band3')].rsr,\
          "name":'s2amsi Band3'},\
         {"x": sensordict['tmodis'][('Band3')].wavelength,"y": sensordict['tmodis'][('Band3')].rsr,\
          "name":'tmodis Band3'},\
         {"x": sensordict['h8ahi'][('Band 2')].wavelength,"y": sensordict['h8ahi'][('Band 2')].rsr,\
          "name": 'h8ahi Band 2'},\
         {"x": sensordict['aster'][('VNIR_Band1')].wavelength,"y": sensordict['aster'][('VNIR_Band1')].rsr, \
          "name": 'astervnir VNIR1'}
        ],
"layout": {"title": "GREEN Relative Spectral Response"}})


1.9.6


In [13]:
# Plot every sensor band for a given sensor
sensor = 'tmodis'
plotlydatalist = []
for key in sensordict[sensor].keys():
    plotlydatalist.append({"x": sensordict[sensor][(key)].wavelength,\
                            "y": sensordict[sensor][(key)].rsr,"name": (key)})
#plotlydatalist
plotly.offline.iplot({ "data": plotlydatalist,"layout": {"title": sensor}})
#plotlydatalist

In [14]:
# input can come in a variety of sampling intervals / regular/ irregular and differing scales
# below allows reshape and interpolate to allow comparison of two spectra

def reshape_interpolate(start, stop, samples, npdatatype, input1dwavelength,input1drsr,wlscalefactor):
    wavelength = np.linspace(start,stop,samples, dtype=float)
    rsr = np.nan_to_num(np.interp(wavelength,input1dwavelength*wlscalefactor, input1drsr))
    return wavelength, rsr

wl, response = reshape_interpolate(300,15000,14200,'float', sensordict['ls7etm']['SWIR(5)-L7'].wavelength ,\
                                   sensordict['ls7etm']['SWIR(5)-L7'].rsr, 1 )
wl2, response2 = reshape_interpolate(300,15000,14200,'float', sensordict['ls8oli']['SWIR1'].wavelength,\
                                     sensordict['ls8oli']['SWIR1'].rsr, 1)

plotly.offline.iplot({
"data": [{"x": wl,"y": response, "name": 'ls7etm Green-L7'}, {"x": wl2,"y": response2, "name": 'ls8oli Pan'},
        ],
"layout": {"title": "Example resampled response"}})


Earth Mover Distance example




In [34]:
sensor1 = 'ls8oli'
sensor2 = 'ls7etm'

#lists to hold inputs to dataframe once we're done
sensor1list = []
sensor1keys = []
sensor2list = []
sensor2keys = []
pcorrelation = []
emdistance = []
weightedcentredelta = []
areadelta = []
fwhmdelta = []

# TODO update the interpolation range to fit the min and max wavelength range for the input pairwise comparison
for key1 in sensordict[sensor1].keys():

    for key2 in sensordict[sensor2].keys():
        sensor1list.append(sensor1)
        sensor1keys.append(key1)
        sensor2list.append(sensor2)
        sensor2keys.append(key2)
        # Find the wavelength range of the rsr values and interpolate within it

        bounds = []
        bounds.append(sensordict[sensor1][(key1)].wavelength\
                      [sensordict[sensor1][(key1)].rsr.replace(0., np.nan).first_valid_index()])
        bounds.append(sensordict[sensor1][(key1)].wavelength\
                      [sensordict[sensor1][(key1)].rsr.replace(0., np.nan).last_valid_index()])
        bounds.append(sensordict[sensor2][(key2)].wavelength\
                      [sensordict[sensor2][(key2)].rsr.replace(0., np.nan).first_valid_index()])
        bounds.append(sensordict[sensor2][(key2)].wavelength\
                      [sensordict[sensor2][(key2)].rsr.replace(0., np.nan).last_valid_index()])
        
        # Interpolate rsr 
        sensor1wl, sensor1rsr = \
        reshape_interpolate(min(bounds),max(bounds),max(bounds)-min(bounds)+1,'float', sensordict[sensor1][(key1)].wavelength,\
                            sensordict[sensor1][(key1)].rsr.replace(0., np.nan), 1)
        sensor2wl, sensor2rsr = \
        reshape_interpolate(min(bounds),max(bounds),max(bounds)-min(bounds)+1,'float', sensordict[sensor2][(key2)].wavelength,\
                            sensordict[sensor2][(key2)].rsr.replace(0., np.nan), 1)
        
        # A smoothed distrubution seems important for Earth Mover Distance
        A = ndimage.filters.gaussian_filter(sensor1rsr, 10)
        B = ndimage.filters.gaussian_filter(sensor2rsr, 10)

        print "Calculating equivalence metrics for :", sensor1, key1, "with", sensor2, key2
        # Earth Mover Distance - has issues with certain S2AMSI bands even though EMD calc completes...
        # From - https://github.com/andreasjansson/python-emd
        try:
            # normalise - confirm with someone who has maths skills that doing this makes sense - seems to be required for EMD
            A = (A - A.min())/(A.max() - A.min())
            B = (B - B.min())/(B.max() - B.min())

            EMD = emd.emd(range(500), range(500), signal.resample(A,500), signal.resample(B,500))
            
            #print "EMD success:", EMD
        except:
            EMD = 0
            pass
        emdistance.append(EMD)
        # Pearson correlation coefficient
        pearson = stats.pearsonr(sensor1rsr, sensor2rsr)
        pcorrelation.append(pearson[0])
        # "Area" under each curve
        sensor1trapz = np.trapz(sensor1rsr, sensor1wl)
        sensor2trapz = np.trapz(sensor2rsr, sensor2wl)
        areadelta.append(abs(sensor1trapz - sensor2trapz))
        sensor1mean = np.average(sensor1wl, weights=sensor1rsr)
        sensor2mean = np.average(sensor2wl, weights=sensor2rsr)
        weightedcentredelta.append(abs(sensor1mean - sensor2mean))
        
        #FWHM and roots
        spline1 = UnivariateSpline(sensor1wl, A-A.max()/2, s=0)
        spline2 = UnivariateSpline(sensor2wl, B-B.max()/2, s=0)
        try:
            sensor1r1, sensor1r2 = spline1.roots()
            sensor2r1, sensor2r2 = spline2.roots()
        except:
            sensor1r1 = 100.
            sensor1r2 = 100. 
            sensor2r1 = 100.
            sensor2r2 = 100. 
            pass
        fwhmdelta.append(abs((sensor1r2-sensor1r1)-(sensor2r2-sensor2r1)))
        
        # Reduce the number of plots output, use a correlation threshold to determine whether to display

        if (pearson[0] > 0.3):
            plotly.offline.iplot({
                "data": [{"x": sensor1wl,"y": sensor1rsr, "name": sensor1+"-"+key1, "line": dict(color = ('rgb(255, 1, 1)'))},\
                         {"x": pandas.Series([sensor1mean, sensor1mean]),"y": pandas.Series([0,1]), "name": 'mean wavelength',\
                          "line": dict(color = ('rgb(255, 1, 1)'), width = 1, dash = 'dash')},\
                         {"x": sensor1wl,"y": A, "name": 'A',\
                          "name": 'gaussian', "line": dict(color = ('rgb(255, 1, 1)'), width = 1, dash = 'dot')},\
                         {"x": pandas.Series([sensor1r1, sensor1r1]),"y": pandas.Series([0,1]), "name": 'fwhm root1',\
                          "line": dict(color = ('rgb(255, 1, 1)'), width = 1, dash = 'dashdot')},
                         {"x": pandas.Series([sensor1r2, sensor1r2]),"y": pandas.Series([0,1]), "name": 'fwhm root2',\
                          "line": dict(color = ('rgb(255, 1, 1)'), width = 1, dash = 'dashdot')},

                         
                         {"x": sensor2wl,"y": sensor2rsr, "name": sensor2+"-"+key2, "line": dict(color = ('rgb(1, 1, 255)'))},\
                         {"x": pandas.Series([sensor2mean, sensor2mean]),"y": pandas.Series([0,1]) , "name": 'mean wavelength',\
                          "line": dict(color = ('rgb(1, 1, 255)'), width = 1, dash = 'dash')},
                         {"x": sensor2wl,"y": B, "name": 'B',\
                          "name": 'gaussian', "line": dict(color = ('rgb(1, 1, 255)'), width = 1, dash = 'dot')},\
                         {"x": pandas.Series([sensor2r1, sensor2r1]),"y": pandas.Series([0,1]), "name": 'fwhm root1',\
                          "line": dict(color = ('rgb(1, 1, 255)'), width = 1, dash = 'dashdot')},
                         {"x": pandas.Series([sensor2r2, sensor2r2]),"y": pandas.Series([0,1]), "name": 'fwhm root2',\
                          "line": dict(color = ('rgb(1, 1, 255)'), width = 1, dash = 'dashdot')},\
                         
                        ],
                    
                "layout": {"title": 'Equivalence match: '+sensor1+' '+key1+' and '+sensor2+' '+key2}})

        del bounds, EMD, sensor1trapz, sensor2trapz, pearson, A, B, sensor1wl, sensor1rsr,sensor2wl, sensor2rsr,\
        sensor1r1, sensor1r2, sensor2r1, sensor2r2

spectrumcomparison = pandas.DataFrame({'sensor1' : sensor1list, 'sensor1keys' : sensor1keys, 'sensor2': sensor2list,\
                                       'sensor2keys': sensor2keys, 'pcorrelation' : pcorrelation, 'distance': emdistance,\
                                       'areadelta': areadelta, 'weightedcentredelta': weightedcentredelta, 'fwhmdelta': fwhmdelta})


Calculating equivalence metrics for : ls8oli Blue with ls7etm Pan-L7
Calculating equivalence metrics for : ls8oli Blue with ls7etm SWIR(5)-L7
Calculating equivalence metrics for : ls8oli Blue with ls7etm Green-L7
Calculating equivalence metrics for : ls8oli Blue with ls7etm Red-L7
Calculating equivalence metrics for : ls8oli Blue with ls7etm NIR-L7
Calculating equivalence metrics for : ls8oli Blue with ls7etm SWIR(7)-L7
Calculating equivalence metrics for : ls8oli Blue with ls7etm Blue-L7


Calculating equivalence metrics for : ls8oli Red with ls7etm Pan-L7
Calculating equivalence metrics for : ls8oli Red with ls7etm SWIR(5)-L7
Calculating equivalence metrics for : ls8oli Red with ls7etm Green-L7
Calculating equivalence metrics for : ls8oli Red with ls7etm Red-L7


Calculating equivalence metrics for : ls8oli Red with ls7etm NIR-L7
Calculating equivalence metrics for : ls8oli Red with ls7etm SWIR(7)-L7
Calculating equivalence metrics for : ls8oli Red with ls7etm Blue-L7
Calculating equivalence metrics for : ls8oli SWIR1 with ls7etm Pan-L7
Calculating equivalence metrics for : ls8oli SWIR1 with ls7etm SWIR(5)-L7


Calculating equivalence metrics for : ls8oli SWIR1 with ls7etm Green-L7
Calculating equivalence metrics for : ls8oli SWIR1 with ls7etm Red-L7
Calculating equivalence metrics for : ls8oli SWIR1 with ls7etm NIR-L7
Calculating equivalence metrics for : ls8oli SWIR1 with ls7etm SWIR(7)-L7
Calculating equivalence metrics for : ls8oli SWIR1 with ls7etm Blue-L7
Calculating equivalence metrics for : ls8oli SWIR2 with ls7etm Pan-L7
Calculating equivalence metrics for : ls8oli SWIR2 with ls7etm SWIR(5)-L7
Calculating equivalence metrics for : ls8oli SWIR2 with ls7etm Green-L7
Calculating equivalence metrics for : ls8oli SWIR2 with ls7etm Red-L7
Calculating equivalence metrics for : ls8oli SWIR2 with ls7etm NIR-L7
Calculating equivalence metrics for : ls8oli SWIR2 with ls7etm SWIR(7)-L7


Calculating equivalence metrics for : ls8oli SWIR2 with ls7etm Blue-L7
Calculating equivalence metrics for : ls8oli CoastalAerosol with ls7etm Pan-L7
Calculating equivalence metrics for : ls8oli CoastalAerosol with ls7etm SWIR(5)-L7
Calculating equivalence metrics for : ls8oli CoastalAerosol with ls7etm Green-L7
Calculating equivalence metrics for : ls8oli CoastalAerosol with ls7etm Red-L7
Calculating equivalence metrics for : ls8oli CoastalAerosol with ls7etm NIR-L7
Calculating equivalence metrics for : ls8oli CoastalAerosol with ls7etm SWIR(7)-L7
Calculating equivalence metrics for : ls8oli CoastalAerosol with ls7etm Blue-L7
Calculating equivalence metrics for : ls8oli Green with ls7etm Pan-L7
Calculating equivalence metrics for : ls8oli Green with ls7etm SWIR(5)-L7
Calculating equivalence metrics for : ls8oli Green with ls7etm Green-L7


Calculating equivalence metrics for : ls8oli Green with ls7etm Red-L7
Calculating equivalence metrics for : ls8oli Green with ls7etm NIR-L7
Calculating equivalence metrics for : ls8oli Green with ls7etm SWIR(7)-L7
Calculating equivalence metrics for : ls8oli Green with ls7etm Blue-L7
Calculating equivalence metrics for : ls8oli Pan with ls7etm Pan-L7
Calculating equivalence metrics for : ls8oli Pan with ls7etm SWIR(5)-L7
Calculating equivalence metrics for : ls8oli Pan with ls7etm Green-L7


Calculating equivalence metrics for : ls8oli Pan with ls7etm Red-L7
Calculating equivalence metrics for : ls8oli Pan with ls7etm NIR-L7
Calculating equivalence metrics for : ls8oli Pan with ls7etm SWIR(7)-L7
Calculating equivalence metrics for : ls8oli Pan with ls7etm Blue-L7
Calculating equivalence metrics for : ls8oli NIR with ls7etm Pan-L7
Calculating equivalence metrics for : ls8oli NIR with ls7etm SWIR(5)-L7
Calculating equivalence metrics for : ls8oli NIR with ls7etm Green-L7
Calculating equivalence metrics for : ls8oli NIR with ls7etm Red-L7
Calculating equivalence metrics for : ls8oli NIR with ls7etm NIR-L7


Calculating equivalence metrics for : ls8oli NIR with ls7etm SWIR(7)-L7
Calculating equivalence metrics for : ls8oli NIR with ls7etm Blue-L7
Calculating equivalence metrics for : ls8oli Cirrus with ls7etm Pan-L7
Calculating equivalence metrics for : ls8oli Cirrus with ls7etm SWIR(5)-L7
Calculating equivalence metrics for : ls8oli Cirrus with ls7etm Green-L7
Calculating equivalence metrics for : ls8oli Cirrus with ls7etm Red-L7
Calculating equivalence metrics for : ls8oli Cirrus with ls7etm NIR-L7
Calculating equivalence metrics for : ls8oli Cirrus with ls7etm SWIR(7)-L7
Calculating equivalence metrics for : ls8oli Cirrus with ls7etm Blue-L7


In [37]:
match_limits = {'pcorrelation': 0.3, 'distance': 50, 'weightedcentredelta': 70, 'areadelta': 100, 'fwhmdelta': 50}
spectrumcomparison.sort_values('distance', ascending=True)


,areadelta,distance,fwhmdelta,pcorrelation,sensor1,sensor1keys,sensor2,sensor2keys,weightedcentredelta
49,291.375935,0.065354,342.019874,0.209937,ls8oli,NIR,ls7etm,Pan-L7,144.576290
53,92.790543,0.403141,92.589442,0.365454,ls8oli,NIR,ls7etm,NIR-L7,30.010239
10,23.223643,2.092045,19.651254,0.659625,ls8oli,Red,ls7etm,Red-L7,6.833346
35,263.295822,6.667982,317.110065,-0.117550,ls8oli,Green,ls7etm,Pan-L7,159.118458
44,83.422875,15.965477,87.238815,0.323567,ls8oli,Pan,ls7etm,Green-L7,30.639492
6,10.903178,21.295927,11.459764,0.826141,ls8oli,Blue,ls7etm,Blue-L7,3.881135
37,21.489599,22.885185,22.287557,0.794450,ls8oli,Green,ls7etm,Green-L7,0.294817
15,106.908539,24.336010,113.450845,0.436584,ls8oli,SWIR1,ls7etm,SWIR(5)-L7,41.182752
26,70.274992,26.104931,94.395452,0.649998,ls8oli,SWIR2,ls7etm,SWIR(7)-L7,6.848835
45,101.026563,35.066589,109.749417,-0.038063,ls8oli,Pan,ls7etm,Red-L7,69.767739


In [38]:
spectrumcomparison[(spectrumcomparison.pcorrelation > match_limits['pcorrelation']) &\
                   (spectrumcomparison.distance < match_limits['distance']) &\
                   (spectrumcomparison.weightedcentredelta < match_limits['weightedcentredelta']) &\
                   (spectrumcomparison.areadelta < match_limits['areadelta']) &\
                   (spectrumcomparison.areadelta < match_limits['fwhmdelta'])
                   ]

,areadelta,distance,fwhmdelta,pcorrelation,sensor1,sensor1keys,sensor2,sensor2keys,weightedcentredelta
6,10.903178,21.295927,11.459764,0.826141,ls8oli,Blue,ls7etm,Blue-L7,3.881135
10,23.223643,2.092045,19.651254,0.659625,ls8oli,Red,ls7etm,Red-L7,6.833346
37,21.489599,22.885185,22.287557,0.794450,ls8oli,Green,ls7etm,Green-L7,0.294817


In [18]:
# Write this puppy out as per Josh's
hdf = pandas.HDFStore('store.h5')
#TODO ... later

In [19]:
# Solar irradiance curve - how much radiant energy is hitting the sensor...proportionally? 
# http://rredc.nrel.gov/solar/spectra/am1.5/
# TODO - done above but difficult to track down the normalised transmission data for a representative atmosphere
# across the 300 to 15000 nanometer spectrum

In [33]:
# TODO matchto_sensor(platform, sensor)  return(platform, sensor, band, wavelenghth_array, rsr_array)
# TODO make_synthetic_band(band_centre_wavelength, FWHM)/
#     http://www.idlcoyote.com/math_tips/normsigma.html
#     returns data['synthetic_platform']['synthetic_band'].wavelength /  rsr representing normal curve  
# TODO matchto_synthetic_band
# TODO return_matching_platform_band(wavelength range in, max)
# ls8oli example
fwhm = 510 - 450
mean = 450 + (fwhm/2)
sigma = fwhm / 2.35

#rsr = signal.gaussian(10000, sigma, sym=False)
rsr = np.random.normal(mean, sigma, 1000)
from scipy.stats import norm
rsr = norm(rsr)
window = signal.gaussian(1000, sigma)

#plt.plot(window)
#plt.title(r"Gaussian window ($\sigma$=7)")
#plt.ylabel("Amplitude")
#plt.xlabel("Sample")

#plt.figure()
#plt.show()

plotly.offline.iplot({
"data": [{"y": rsr, "name": 'synthetic_band'}, 
        ],
"layout": {"title": "Synthetic Band Response"}})

TypeError: <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6b134d0a90> is not JSON serializable

In [21]:
# TODO Apply the match criteria and plot the pairs

In [22]:
# TODO For a given band, find the matching bands from all other sensors and plot them together

In [23]:
# TODO Band resampling to provide proportional inclusion of multiple summed bands in synthetic band (i.e. with pan band,
# or hyperspectral, narrow bands to broad band)

In [24]:
# Spectral library stuff for much later - http://speclab.cr.usgs.gov/ 

In [25]:
# Some bandpass and fwhm ...create synth band possibilities here - http://ssb.stsci.edu/pysynphot/docs/